In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import wavfile
from scipy import signal
from scipy import stats
from pyorderedfuzzy.ofcandles import ofcandle
from pyorderedfuzzy.ofnumbers import ofnumber
import prepare_data as prep
%matplotlib tk
import IPython.display as ipd

In [2]:
ftrain = pd.read_pickle('../data/ftrain.pkl')
ftrain.head()

,folder,file,label
0,bed,00176480_nohash_0.wav,unknown
1,bed,004ae714_nohash_0.wav,unknown
2,bed,004ae714_nohash_1.wav,unknown
3,bed,00f0204f_nohash_0.wav,unknown
4,bed,00f0204f_nohash_1.wav,unknown


In [3]:
df_folder = ftrain.groupby(['folder'])
samples_path = {}
for i, folder in df_folder:
    sample = folder.sample(1)
    samples_path[i] = '../data/train/audio/'+str(sample['folder'].min())+'/'+str(sample['file'].min())
samples_path

{'bed': '../data/train/audio/bed/b21f0fa4_nohash_0.wav',
 'bird': '../data/train/audio/bird/f6af2457_nohash_0.wav',
 'cat': '../data/train/audio/cat/cfde27ba_nohash_0.wav',
 'dog': '../data/train/audio/dog/61e50f62_nohash_0.wav',
 'down': '../data/train/audio/down/67961766_nohash_0.wav',
 'eight': '../data/train/audio/eight/c1d39ce8_nohash_3.wav',
 'five': '../data/train/audio/five/f44f440f_nohash_1.wav',
 'four': '../data/train/audio/four/acfd3bc3_nohash_0.wav',
 'go': '../data/train/audio/go/31e686d2_nohash_0.wav',
 'happy': '../data/train/audio/happy/cc2f1a85_nohash_0.wav',
 'house': '../data/train/audio/house/27b9a84f_nohash_0.wav',
 'left': '../data/train/audio/left/ffd2ba2f_nohash_3.wav',
 'marvin': '../data/train/audio/marvin/7bf801d1_nohash_1.wav',
 'nine': '../data/train/audio/nine/135c6841_nohash_0.wav',
 'no': '../data/train/audio/no/e41e41f7_nohash_0.wav',
 'off': '../data/train/audio/off/80c45ed6_nohash_0.wav',
 'on': '../data/train/audio/on/f3d06008_nohash_3.wav',
 'one':

In [16]:
def moving_average(data, period):
    smooth = np.zeros(len(data))
    for i in range(len(data)):
        smooth[i]=data[i]
        it = 1
        for j in range(1, period):
            if i-j>=0:
                smooth[i]+=data[i-j]
                it+=1
        smooth[i]=smooth[i]/it
    return smooth



In [108]:
sample = ftrain.sample(1)
sample_path = '../data/train/audio/'+str(sample['folder'].min())+'/'+str(sample['file'].min())
#sample_path = '../data/train/audio/two/d0faf7e4_nohash_4.wav'
print(sample_path)
silence_level = 0.052
rate, data = wavfile.read(sample_path)
norm_data = (data-np.mean(data))/np.max(np.abs(data))
frequencies, times, spectogram = signal.spectrogram(norm_data, rate)
analytic_signal = signal.hilbert(norm_data)
amplitude_envelope = np.abs(analytic_signal)
smooth_amplitude = moving_average(amplitude_envelope, 11)
last = np.argwhere(smooth_amplitude>silence_level)[-1][0]
smooth_amplitude_back = moving_average(amplitude_envelope[::-1], 11)
first = np.argwhere(smooth_amplitude_back[::-1]>silence_level)[0][0]

../data/train/audio/zero/51f4d5b0_nohash_0.wav


In [111]:
fig = plt.figure(figsize = (14,4))
ax1 = fig.add_subplot(111)
ax1.set_title('signal - amplitude')
ax1.plot(norm_data)
ax1.plot(amplitude_envelope)
ax1.plot(smooth_amplitude)
ax1.plot(smooth_amplitude_back[::-1])
ax1.axhline(0.051)
ax1.axhline(-0.051)
ax1.axvline(first, c='r')
ax1.axvline(last, c='g')
plt.show()

In [110]:
ipd.Audio(data[first:last], rate=rate)

In [65]:
from pydub import AudioSegment, silence
song = AudioSegment.from_wav("../data/train/audio/stop/977a3be4_nohash_0.wav")

In [60]:
silence.detect_nonsilent(song)

[[0, 836]]

In [63]:
song.rms

2519

In [67]:
song[1:100]

In [55]:
a.sample_width

2

In [56]:
song.sample_width

2

In [81]:
a = silence.split_on_silence(song, silence_thresh=0)

In [82]:
a

In [78]:
a[0]

In [72]:
20*np.log10(0.05)

-26.020599913279625